# SymboleoNLP Demo

This is a demo notebook that illustrates a very basic example of the functionality of the SymboleoNLP tool. 

The user can enter input that is based on a controlled contract language. This input represents a refinement on a given contract obligation. The user refines the obligation using the controlled natural language, and the tool will update the corresponding Symboleo template accordingly.

To run the notebook, simply run the 'Import NLP Tools' section, which can take a minute to install. Then run all of the pre-execution code, which sets up the classes and helper functionality. Then run the code cell by cell under the 'Execution' section. Follow the prompts and enter the appropriate input to customize the sample obligation.



NOTE: This is a very basic demo that only covers a small slice of the functionality and the Symboleo specification. There is therefore no sophisticated error handling and the demo will not easily scale beyond the examples provided.

# Import NLP Tools

In [192]:
# Setup
!pip install mlconjug3

!python -m spacy download en_core_web_md

In [194]:
import mlconjug3

from mlconjug3 import Conjugator

import spacy

nlp = spacy.load('en_core_web_md')
conjugator = Conjugator(language = 'en')

# Pre-Execution

## Symboleo Spec

In [288]:
class SymEvent:
    def to_sym(self):
        raise NotImplementedError()

class VariableEvent(SymEvent):
    def __init__(self, name: str):
        self.name = name

    def to_sym(self):
        return f'{self.name}'

class SymPoint:
    def to_sym(self):
        raise NotImplementedError()

class PointExpression():
    # PointFunction | PointAtom
    def to_sym(self):
        raise NotImplementedError()

class PointAtom(PointExpression):
    def to_sym(self):
        raise NotImplementedError()


class PointVDE(PointAtom):
    def __init__(self, name: str = ''):
        self.name = name
    
    def to_sym(self):
        return self.name

class Point(SymPoint):
    point_expression = PointExpression()

    def __init__(self, point_expression: PointExpression):
        self.point_expression = point_expression
    
    def to_sym(self):
        return self.point_expression.to_sym()


class PointVDE(SymPoint):
    def __init__(self, name: str = ''):
        self.name = name
    
    def to_sym(self):
        return self.name

class PointFunction(SymPoint):
    arg = PointAtom()

    def __init__(self, arg: PointAtom, time_value: str, time_unit: str):
        self.name = 'Date.add' 
        self.arg = arg
        self.time_value = time_value
        self.time_unit = time_unit

    def to_sym(self):
        return f'{self.name}({self.arg.to_sym()}, {self.time_value}, {self.time_unit})'

In [289]:
class PredicateFunction:
    def to_sym(self):
        raise NotImplementedError()


class Happens(PredicateFunction):
    event = SymEvent()

    def __init__(self, event: SymEvent):
        self.event = event
    
    def to_sym(self):
        return f'Happens({self.event.to_sym()})'


class HappensBefore(PredicateFunction):
    event = SymEvent()
    point = SymPoint()

    def __init__(self, event: SymEvent, point: SymPoint):
        self.name = 'happensBefore'
        self.event = event
        self.point = point
    
    def to_sym(self):
        return f'{self.name}({self.event.to_sym()}, {self.point.to_sym()})'


class HappensBeforeEvent(PredicateFunction):
    event1 = SymEvent()
    event2 = SymEvent()

    def __init__(self, event1: SymEvent, event2: SymEvent):
        self.name = 'HappensBeforeEvent'
        self.event1 = event1
        self.event2 = event2
    
    def to_sym(self):
        return f'{self.name}({self.event1.to_sym()}, {self.event2.to_sym()})'



In [290]:
class Obligation:
    def __init__(
        self,
        id: str,
        trigger: PredicateFunction,
        debtor: str,
        creditor: str,
        antecedent: PredicateFunction,
        consequent: PredicateFunction
    ):
        self.id = id
        self.trigger = trigger
        self.debtor = debtor
        self.creditor = creditor
        self.antecedent = antecedent
        self.consequent = consequent
    

    def update(self, str_component: str, predicate: PredicateFunction):
        setattr(self, str_component, predicate)


    def get_default_event(self, str_component:str):
        component: PredicateFunction = getattr(self, str_component)
        if not component:
            return None
        if isinstance(component, Happens):
            return component.event


    def to_sym(self):
        trigger_text = ''
        if self.trigger:
            trigger_text = self.trigger.to_sym() + ' -> '

        deb_text = self.debtor
        cred_text = self.creditor
        if self.antecedent:
            ant_text = self.antecedent.to_sym()
        else:
            ant_text = 'T'        
        con_text = self.consequent.to_sym()

        return f'{self.id}: {trigger_text}O({deb_text}, {cred_text}, {ant_text}, {con_text});'


In [291]:
class Basket:
    default_event: SymEvent
    initial_norm: Obligation

## Frames

In [292]:
import copy

class Frame:
    pattern: List[str]
    op_code: str # Change this

    def to_text(self) -> str:
        raise NotImplementedError()
    
    def is_complete(self) -> bool:
        return False
    
    @staticmethod
    def copy(frame: Frame):
        return copy.deepcopy(frame)


class BeforeDateFrame(Frame):
    pattern = ['ROOT', 'BEFORE', 'DATE']
    op_code = 'REFINE_PREDICATE'
    date_text: str = ''

    def is_complete(self):
        return self.date_text != ''

    def to_text(self):
        return f'before {self.date_text}'


class BeforeEventFrame(Frame):
    pattern = ['ROOT', 'BEFORE', 'EVENT']
    op_code = 'REFINE_PREDICATE'
    subject: str = ''
    verb: str = ''
    d_object: str = ''

    def is_complete(self):
        return self.subject != '' and self.verb != '' 

    def to_text(self):
        raw_verb = self.verb
        doc = nlp(raw_verb)
        lemma = doc[0].lemma_
        verb = conjugator.conjugate(lemma)
        conj_verb = verb["indicative"]["indicative present"]["he/she/it"]
        return f'before {self.subject} {conj_verb} {self.d_object}'
    

class IfEventFrame(Frame):
    pattern = ['ROOT', 'IF', 'EVENT']
    op_code = 'ADD_TRIGGER'
    subject: str = ''
    verb: str = ''
    d_object: str = ''

    def is_complete(self):
        return self.subject != '' and self.verb != '' 

    def to_text(self):
        raw_verb = self.verb
        doc = nlp(raw_verb)
        lemma = doc[0].lemma_
        verb = conjugator.conjugate(lemma)
        conj_verb = verb["indicative"]["indicative present"]["he/she/it"]
        return f'before {self.subject} {conj_verb} {self.d_object}'


class WithinTimespanEventFrame(Frame):
    pattern = ['ROOT', 'WITHIN', 'TIMESPAN', 'EVENT']
    op_code = 'REFINE_PREDICATE'
    timespan: str = ''
    subject: str = ''
    verb: str = ''
    d_object: str = ''

    def is_complete(self):
        return self.subject != '' and self.verb != '' and self.timespan != ''

    def to_text(self):
        raw_verb = self.verb
        doc = nlp(raw_verb)
        lemma = doc[0].lemma_
        verb = conjugator.conjugate(lemma)
        conj_verb = verb["indicative"]["indicative present continuous"]["he/she/it"]
        return f'within {self.timespan} of {self.subject} {conj_verb} {self.d_object}'


def get_all_frames():
    return  [
        BeforeDateFrame(),
        BeforeEventFrame(),
        IfEventFrame(),
        WithinTimespanEventFrame()
    ]
    

## Tokens

In [309]:
from __future__ import annotations
from typing import List

class AbstractNode:
    id: str = None
    node_type: str = None
    prompt: str = None
    children: List[AbstractNode] = []
    value = None
    child: AbstractNode = None

    def __init__(self, id: str, children: List[AbstractNode] = []):
        self.id = id
        self.children = children
    
    def get_value(self):
        raise NotImplementedError()
    
    def build_frame(self, frame: Frame) -> Frame:
        return frame

    def to_obj(self, basket = None):
        raise NotImplementedError()


class RootNode(AbstractNode):
    def __init__(self, id: str, children: List[AbstractNode] = []):
        super().__init__(id, children)
        self.prompt = None
        self.node_type = 'ROOT'

    def get_value(self):
        return None
    
    def to_obj(self, basket):
        return self.child.to_obj(basket)


class BeforeNode(AbstractNode):
    def __init__(self, id: str, children: List[AbstractNode] = []):
        super().__init__(id, children)
        self.prompt = 'before'
        self.node_type = 'BEFORE'
        self.value = 'before'
    
    def get_value(self):
        return 'before'
    
    def to_obj(self, basket):        
        if self.child.node_type == 'EVENT':
            event2 = self.child.to_obj(basket)
            return HappensBeforeEvent(basket.default_event, event2)

        elif self.child.node_type == 'DATE':
            p = self.child.to_obj(basket)
            return HappensBefore(basket.default_event, p)

        raise NotImplementedError('Oops!')


class WithinNode(AbstractNode):
    def __init__(self, id: str, children: List[AbstractNode] = []):
        super().__init__(id, children)
        self.prompt = 'within'
        self.node_type = 'WITHIN'
        self.value = 'within'

    def get_value(self):
        return 'within'

    def to_obj(self, basket):
        time_info = self.child.to_obj(basket)
        event2 = self.child.child.to_obj(basket)
        p = PointFunction(event2, time_info[0], time_info[1])
        return HappensBefore(basket.default_event, p)


class IfNode(AbstractNode):
    def __init__(self, id: str, children: List[AbstractNode] = []):
        super().__init__(id, children)
        self.prompt = 'if'
        self.node_type = 'IF'
        self.value = 'if'
    
    def get_value(self):
        return 'if'
    
    def to_obj(self, basket):
        if self.child.node_type == 'EVENT':
            evt = self.child.to_obj(basket)    
            return Happens(evt)

        raise NotImplementedError('Oops!')
    


class TimespanNode(AbstractNode):
    def __init__(self, id: str, children: List[AbstractNode] = []):
        super().__init__(id, children)
        self.prompt = 'Enter a timespan'
        self.node_type = 'TIMESPAN'

    def get_value(self):
        result = input('Enter a timespan (e.g. 2 weeks): ')
        self.value = result
        return result
    
    def build_frame(self, frame: Frame) -> Frame:
        new_frame = Frame.copy(frame)
        if isinstance(new_frame, WithinTimespanEventFrame):
            new_frame.timespan = f'{self.value}'
        return new_frame


    def to_obj(self, basket):
        return self.value.split(' ')


class EventNode(AbstractNode):
    def __init__(self, id: str, children: List[AbstractNode] = []):
        super().__init__(id, children)
        self.prompt = 'Describe an event'
        self.node_type = 'EVENT'

    def get_value(self):
        agent = input('Enter an subject: ')    
        verb = input('Enter a verb: ')
        obj = input('Enter an optional direct object: ')
        self.value = (agent, verb, obj)
        return (agent, verb, obj)
    

    def build_frame(self, frame: Frame) -> Frame:
        new_frame = Frame.copy(frame)

        if isinstance(new_frame, BeforeEventFrame) or \
            isinstance(new_frame, WithinTimespanEventFrame) or \
            isinstance(new_frame, IfEventFrame):
            new_frame.subject = self.value[0]
            new_frame.verb = self.value[1]
            new_frame.d_object = self.value[2]
        
        return new_frame

    def to_obj(self, basket):
        if self.value[2]:
            evt_value = f'evt_{self.value[1]}({self.value[0]}, {self.value[2]})'
        else:
            evt_value = f'evt_{self.value[1]}({self.value[0]})'
        return VariableEvent(evt_value)


class DateNode(AbstractNode):
    def __init__(self, id: str, children: List[AbstractNode] = []):
        super().__init__(id, children)
        self.prompt = 'Enter a date'
        self.node_type = 'DATE'

    def get_value(self):
        result = input('Enter a date (yyyy/mm/dd): ')
        self.value = result
        return result

    def build_frame(self, frame: Frame) -> Frame:
        new_frame = Frame.copy(frame)

        if isinstance(new_frame, BeforeDateFrame):
            new_frame.date_text = self.value
        
        return new_frame


    def to_obj(self, basket):
        return PointVDE(self.value)




## Processing

In [294]:
def generate_grammar():
    date_node = DateNode('Date')
    event_node = EventNode('Event')
    timespan_node = TimespanNode('Timespan', [event_node])
    if_node = IfNode('If', [event_node])
    within_node = WithinNode('Within', [timespan_node])
    before_node = BeforeNode('Before', [event_node, date_node])
    root_node = RootNode('Root', [before_node, within_node, if_node])
    return root_node 

In [295]:
def select_grammar(node: RootNode):
    result = []

    target = node

    while (len(target.children) > 0):
        print('\nChoose an option:')
        children_dict = {i+1: target.children[i] for i in range(0, len(target.children))}

        # User selects next child
        for ci in children_dict:
            cn = children_dict[ci]
            print('-', ci, cn.prompt)

        k = input("Enter target id: ")
        target = children_dict[int(k)]
        
        value = target.get_value()
        result.append(target)
    
    for i,x in enumerate(result[:-1]):
        x.child = result[i+1] 

    return result

In [296]:
def sublist_finder(a, b):
    for idx in range(len(a) - len(b) + 1):
        if a[idx: idx + len(b)] == b:
            return True

    return False


def inner_frame_checker(node_list: List[AbstractNode], pattern: List[str]):
    if len(node_list) < len(pattern):
            return True
        
    node_list_types = [x.node_type for x in node_list]
    return sublist_finder(node_list_types, pattern)


def build_frame(frame: Frame, node_list: List[AbstractNode]):
    new_frame = copy.deepcopy(frame)
    for node in node_list:
        new_frame = node.build_frame(new_frame)
    
    return new_frame


def check_frames(node_list: List[AbstractNode]) -> List[Frame]:
    results: List[Frame] = []
    all_frames = get_all_frames()

    for frame in all_frames:
        if inner_frame_checker(node_list, frame.pattern):
            next_frame = build_frame(frame, node_list)

            if next_frame.is_complete():
                results.append(next_frame)
    
    return results




In [297]:
def get_component(frame: Frame):
    if frame.op_code == 'REFINE_PREDICATE':
        return 'consequent'
    else:
        return 'trigger'

In [298]:
class NlSym:
    def __init__(self, nl:str, sym: Obligation):
        self.nl = nl
        self.sym = sym

In [299]:
def generate_result(init_obj: NlSym, node_list: List[AbstractNode]):
    frames = check_frames(selection)
    frame = frames[0]
    nl_parm = frame.to_text()
    new_nl = init_obj.nl.replace('[PARAMETER]', nl_parm)
    component = get_component(frame)

    basket = Basket()
    new_sym = copy.deepcopy(init_obj.sym)
    basket.default_event = new_sym.get_default_event(component)
    basket.initial_norm = new_sym

    new_pred = selection[0].to_obj(basket)

    # update norm

    new_sym.update(component, new_pred)
    
    return NlSym(new_nl, new_sym)



# Execution

### Initial contract:
- NL: The seller shall deliver the goods to the buyer [PARAMETER]
- SYM: test_ob: O(seller, buyer, T, Happens(evt_delivered));

### Test cases:

#### Within TIMESPAN of EVENT:
- Input Sequence: 2 (within), 1 (timespan), 2 weeks (timespan value), 1 (event), contract (subject), terminate (verb), [BLANK] (object)
- Updated NL: The seller shall deliver the goods to the buyer within 2 weeks of contract terminating 
- Updated SYM: test_ob: O(seller, buyer, T, happensBefore(evt_delivered, Date.add(evt_terminate(contract), 2, weeks)));

#### Before EVENT
- Input Sequence: 1 (before), 1 (event), buyer (subject), pay (verb), seller (object)
- Updated NL: The seller shall deliver the goods to the buyer before buyer pays seller
- Updated SYM: test_ob: O(seller, buyer, T, HappensBeforeEvent(evt_delivered, evt_pay(buyer, seller)));

#### Before DATE
- Input Sequence: 1 (before), 2 (date), 2023/03/30 (date value)
- Updated NL: The seller shall deliver the goods to the buyer before 2023/03/30
- Updated SYM: test_ob: O(seller, buyer, T, happensBefore(evt_delivered, 2023/03/30));

In [306]:
sample_text = 'The seller shall deliver the goods to the buyer [PARAMETER]'

symboleo_ob = Obligation('test_ob', None, 'seller', 'buyer', None, Happens(VariableEvent('evt_delivered')))

init_obj = NlSym(sample_text, symboleo_ob)

print(f'NL: {sample_text}')
print(f'SYM: {symboleo_ob.to_sym()}')

NL: The seller shall deliver the goods to the buyer [PARAMETER]
SYM: test_ob: O(seller, buyer, T, Happens(evt_delivered));


In [307]:
root_node = generate_grammar()
selection = select_grammar(root_node)


Choose an option:
- 1 before
- 2 within
- 3 if
Enter target id: 2

Choose an option:
- 1 Enter a timespan
Enter target id: 1
Enter a timespan (e.g. 2 weeks): 2 weeks

Choose an option:
- 1 Describe an event
Enter target id: 1
Enter an agent (seller, buyer): contract
Enter a verb: terminate
Enter an optional object: 


In [308]:
result = generate_result(init_obj, selection)

print(f'Updated NL: {result.nl}')
print(f'Updated SYM: {result.sym.to_sym()}')

Updated NL: The seller shall deliver the goods to the buyer within 2 weeks of contract terminating 
Updated SYM: test_ob: O(seller, buyer, T, happensBefore(evt_delivered, Date.add(evt_terminate(contract), 2, weeks)));
